In [1]:
from tamalero.utils import get_kcu
kcu_ip = "192.168.0.10" ## If your KCU ip is diff, modify it.

kcu = get_kcu(
    kcu_ip,
    control_hub=True,
    host='localhost',
    verbose=False
)
print("Successfully connected to KCU.")

##Check the KCU's status and firmware
kcu.status() # Prints LpGBT link statuses from KCU 
fw_ver = kcu.get_firmware_version(verbose=True) #
kcu.check_clock_frequencies(verbose=True) # Verifies KCU clock stability

##Perform a simple loopback register test to confirm communication
loopback_val = 0xABCD1234
kcu.write_node("LOOPBACK.LOOPBACK", loopback_val) #
read_val = kcu.read_node("LOOPBACK.LOOPBACK").value()
if read_val == loopback_val:
    print(f"KCU Loopback test PASSED: Wrote 0x{loopback_val:X}, Read 0x{read_val:X}")
else:
    print(f"KCU Loopback test FAILED: Wrote 0x{loopback_val:X}, Read 0x{read_val:X}")

IPBus address: chtcp-2.0://localhost:10203?target=192.168.0.10:50001
KCU firmware version: 0.0.0
Successfully connected to KCU.
LPGBT Link Status from KCU:
0x2021  r       READOUT_BOARD_0.LPGBT.DOWNLINK.READY              0x00000001
0x2001  r       READOUT_BOARD_0.LPGBT.UPLINK_0.READY              0x00000001
0x2001  r       READOUT_BOARD_0.LPGBT.UPLINK_0.FEC_ERR_CNT        0x00000000
Firmware version: 2025/07/01 27:20:20 v0.0.0 sha=aa38a3c
0x100B  r       FW_INFO.CLK125_FREQ                               0x07735940
0x1008  r       FW_INFO.CLK320_FREQ                               0x131C88B2
0x1007  r       FW_INFO.CLK_40_FREQ                               0x02639115
0x1009  r       FW_INFO.REFCLK_FREQ                               0x131C88B2
0x1018  r       FW_INFO.RXCLK0_FREQ                               0x131C88B2
0x100E  r       FW_INFO.TXCLK0_FREQ                               0x131C88B2
0x100F  r       FW_INFO.TXCLK1_FREQ                               0x131C88B2
KCU Loopback test

In [ ]:
from tamalero.ReadoutBoard import ReadoutBoard

rb = ReadoutBoard(
    rb=0,
    kcu=kcu,
    config="default", 
    trigger=True,     
    verbose=False,
    allow_bad_links=True
)

# print(f"Readout Board version detected: {rb.ver}")
# print("\nChecking DAQ LpGBT base configuration:")
# rb.DAQ_LPGBT.read_base_config() # check lpgbt register matched lpgbt.yaml or not

# Read on-board temperatures
# print("\nReading temperatures:")
# rb.read_temp(verbose=True) #

# print("\nReading DAQ LpGBT ADCs:")
# try:
#     rb.DAQ_LPGBT.read_adcs(check=True, strict_limits=True) #
# except ValueError as e:
#     print(f"LpGBT ADC check issue: {e}")

# print(f"\nDAQ LpGBT Link Good: {rb.DAQ_LPGBT.link_status()}") 
# fec_errors = rb.get_FEC_error_count() # Reads KCU registers defined in READOUT_BOARD.xml
# print(f"FEC Errors: DAQ LpGBT = {fec_errors.get('DAQ', 'N/A')}")


rb.TRIG_LPGBT.calibrate_adc()
# rb.TRIG_LPGBT.apply_adc_calibration = lambda val : val
rb.DAQ_LPGBT.read_adcs(check=True, strict_limits=True) #
# rb.TRIG_LPGBT.read_adcs(check = True, strict_limits=False)
# rb.connect_modules(verbose = True)


0x1d7 readback value is 0xae
lpGBT version found True
 > lpGBT v2 detected
 > VTRx+ version detected: production
 > Found trigger lpGBT, will configure it now.
 > Enabling VTRX channel for trigger lpGBT
Trigger link does not have a stable connection. Ignoring.


In [18]:
# rb.TRIG_LPGBT.read_differential_adc_raw
gain, offset = rb.TRIG_LPGBT.calibrate_adc(recalibrate=True)
print(gain, offset)
for channel in [0x0, 0x2, 0x4, 0x6]:
    # print(rb.TRIG_LPGBT.read_adc_raw_diff(channel))
    print(((rb.TRIG_LPGBT.read_adc_raw_diff(channel)-offset)/1023*1.5/0.1), (rb.TRIG_LPGBT.read_adc_raw(channel)-rb.TRIG_LPGBT.read_adc_raw(channel+1))/1023*1.5/0.1)
    # print(((rb.TRIG_LPGBT.read_adc_raw(channel))/1023*1.5), ((rb.TRIG_LPGBT.read_adc_raw(channel+1))/1023*1.5))


Recalibrating
1.87890625
Calibrated Trigger ADC. Gain: 1.878906 / Offset: 510
Chip BAAEA82A
Calibration data saved to lpgbt_adc_calibrations.json
1.87890625 510
0.45454545454545453 0.45454545454545453
0.45454545454545453 0.42521994134897356
0.4398826979472141 0.4105571847507331
0.45454545454545453 0.4398826979472141


In [13]:

rb.TRIG_LPGBT.calibrate_adc()
# rb.TRIG_LPGBT.apply_adc_calibration = lambda val : val
# rb.DAQ_LPGBT.read_adcs(check=True, strict_limits=True) #
rb.TRIG_LPGBT.read_adcs(check = True, strict_limits=False)
# rb.connect_modules(verbose = True)

┌─────────────────┬───────┬─────────────────┬───────────────────┬────────────────────┬──────────────────┬─────────────────────┬──────────┬──────────────────────────────────────────────────────┐
│ Register        │ Pin   │   Reading (raw) │   Reading (calib) │   Voltage (direct) │   Voltage (conv) │   current_value (A) │ Status   │ Comment                                              │
├─────────────────┼───────┼─────────────────┼───────────────────┼────────────────────┼──────────────────┼─────────────────────┼──────────┼──────────────────────────────────────────────────────┤
│ Vref1m          │ 2     │             837 │          851.078  │          0.831943  │         1.03743  │                     │ OK       │ Vref from ETROC B1 (B2)                              │
│ Vref2m          │ 6     │             838 │          852.094  │          0.832936  │         1.03867  │                     │ OK       │ Vref from ETROC B2 (C1)                              │
│ Vref3m          │ 4     │   